
# COGS 189: Depression

## Overview
In this project, we ______________.<br>

### EEG Data
For this dataset, we will be using data collected from James F. Cavanagh & John J.B.Allen's Depression Rest (d003) located at the following URL: http://predict.cs.unm.edu/downloads.php.
<br>
***
## Section 1: Setup
We will be importing the following packages: <br>
- numpy
- scipy
- matplotlib
- seaborn
- pandas

In [1]:
import numpy as np                                      # for dealing with data
from scipy.signal import butter, sosfiltfilt, sosfreqz  # for filtering
from scipy.io import loadmat                            # for importing matlab files
import matplotlib.pyplot as plt                         # for plotting
import seaborn as sns                                   # for visualization
import pandas as pd                                     # for importing datasets and handling data

***
## Section 2: Data Cleaning
The dataset we used was broken down into individual MATLAB (.mat) files for each participant. We used MATLAB to convert the files into Comma-separated value (.csv) format. We decided to keep only the EEG data and times for each participant. The files are named according to the participant ID. EEG data files are named only by the participant ID (e.g., "509.csv"), EEG times files have 't' appended to the filename (e.g., "509t.csv").

In [2]:
data_df = pd.read_csv('Data/509_data_6.csv')
times_df = pd.read_csv('Data/509t.csv')

Since each column in the EEG data corresponds to the columns in EEG times, we will use times_df as the column labels for data_df.

In [3]:
data_df.columns=[times_df]

data_df.head()

,0,2,4,6,8,10,12,14,16,18,...,506888,506890,506892,506894,506896,506898,506900,506902,506904,506906
0,-42.48284,-46.31948,-51.36037,-54.47553,-54.11464,-53.90202,-54.85407,-54.40303,-53.71802,-54.37649,...,0,0,0,0,0.0000,9.71704,0.000000,0.000000,0.00,2.452983
1,48.48675,44.48138,42.58455,43.28513,44.06726,41.55170,39.44033,42.32360,43.60525,40.57979,...,0,0,0,0,0.0000,0.00000,0.000148,9.717040,0.00,0.000000
2,-65.25048,-66.15553,-68.49241,-71.37434,-73.33966,-73.69581,-73.61456,-74.31854,-74.32522,-72.89500,...,0,0,0,0,2617.4090,0.00000,0.000000,0.709918,46527.58,2.116113
3,16.57424,15.38467,14.22920,12.94236,12.76103,12.58444,12.35581,12.87979,12.28286,10.40160,...,0,0,0,0,175.3732,34314.64000,0.000000,2.812390,0.00,0.000000
4,-120.97400,-123.59530,-128.45940,-130.62880,-130.72660,-132.59780,-135.11340,-134.28500,-128.96830,-124.42780,...,0,0,0,0,0.0000,0.00000,0.000000,0.000000,0.00,0.000000


***
## Section 3: Data Filtering
In this section, we focus on reformatting and processing the data so that we can have an array of alpha powers for each participant as well as a label (1 if depressed and 0 if otherwise) to signify if a participant was labelled as depressed or not depending on their Beck Depression Test score (> 13).

First, we read in each participant's EEG data as a .csv file (e.g. 509_data.csv) and convert it to a dataframe to process the data later on. We also load in the data for the EEG channels for each person (e.g. participant 509c.csv) and the event timestamps for the participant (e.g. 509e.csv).

In [ ]:
num = '509'
data_t = pd.read_csv('Data/' + num +'_data.csv')
channels = pd.read_csv('Data/' + num + 'c.csv')
events = pd.read_csv('Data/' + num + 'e.csv')
events['type'] = events['type'].astype(str)

In [ ]:
data_t

Participants were directed to keep their eyes open and close their eyes in ~33 seconds. The events included types 1 to 6 and 11 to 16, where triggers were produced at types 11 to 16. In order to filter the data we will only look at types 11, 13, 15  for events with eyes closed and types 12, 14, 16 for events with open eyes.

In [ ]:
eyes_closed = [ '11', '13', '15']
eyes_open = ['12', '14', '16']
fs = 500

The types of events represent the state of the participant(eyes closed/open) and latency indicates the time stamp corresponding for each event. Using the time in events we retrieved the corresponding data for O1, Oz, O2 electrodes for eyes closed or open.

In [ ]:
#Get relevent data from events
#TAs code

#eyes closed
dataset_closed_0 = []
dataset_closed_1 = []
dataset_closed_2 = []

for e in range(1, len(events)):
    if events['type'][e] in eyes_closed:
        time = int(events['latency'][e])
        if time % 2 == 1:
            time = time - 1
        
        loc_1 = data_t.columns.get_loc(str(time))
        
        dataset_closed_0.append(data_t.iloc[0, loc_1])
        dataset_closed_0.append(data_t.iloc[1, loc_1])
        dataset_closed_0.append(data_t.iloc[2, loc_1])
        

#eyes open
dataset_open_0 = []
dataset_open_1 = []
dataset_open_2 = []

for e in range(1, len(events)):
    if events['type'][e] in eyes_open:
        time = int(events['latency'][e])
        if time % 2 == 1:
            time = time - 1
        
        loc_1 = data_t.columns.get_loc(str(time))
        
        dataset_open_0.append(data_t.iloc[0, loc_1])
        dataset_open_0.append(data_t.iloc[1, loc_1])
        dataset_open_0.append(data_t.iloc[2, loc_1])
        

Next, we will plot the raw data for the events where participants closed their eyes and opened their eyes. On the x-axis, we have the data samples for the EEG waves recorded and on the y-axis, we have the amplitude of the EEG waves.

In [ ]:
closed_data = dataset_closed_0
open_data = dataset_open_0

In [ ]:
#Eyes closed
a = np.transpose(closed_data)
plt.plot(a, label = "raw eeg", color='navy');
plt.xlabel('Samples');
plt.ylabel('Amplitude (uV)');
plt.grid(True);
plt.legend(loc = 'best');

In [ ]:
#Open Eyes
a = np.transpose(open_data)
plt.plot(a, label = "raw eeg", color='limegreen');
plt.xlabel('Samples');
plt.ylabel('Amplitude (uV)');
plt.grid(True);
plt.legend(loc = 'best');

Then, we attempted to analyze the data using Fourier Transforms in order to calculate the mean band powers for theta, alpha, beta, and gamma. By getting the values after the Fourier Transformation, we were able to obtain the frequencies at which the events occurred and then look for the values that were within 8 to 12Hz in order to extract the average alpha power for each participant during the events with their eyes closed and eyes open.

In [ ]:
data = closed_data

# Get real amplitudes of FFT (only in postive frequencies)
fft_vals = np.absolute(np.fft.rfft(data))
# Get frequencies for amplitudes in Hz
fft_freq = np.fft.rfftfreq(len(closed_data), 1/fs)


# Define EEG bands
eeg_bands = {'Theta': (4, 8),
             'Alpha': (8, 12),
             'Beta': (12, 30),
             'Gamma': (30, 45)}

# Take the max of the fft amplitude for each EEG band
eeg_band_fft = dict()
for band in eeg_bands: 
    try:
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq < eeg_bands[band][1]))[0]
        eeg_band_fft[band] = np.mean(fft_vals[freq_ix])
    except ValueError:  #raised if `y` is empty.
        print("entered")
        pass

data = open_data

# Get real amplitudes of FFT (only in postive frequencies)
fft_vals = np.absolute(np.fft.rfft(data))
# Get frequencies for amplitudes in Hz
fft_freq = np.fft.rfftfreq(len(data), 1.0/fs)

# Take the max of the fft amplitude for each EEG band
eeg_band_fft_open = dict()
for band in eeg_bands: 
    try:
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq < eeg_bands[band][1]))[0]
        eeg_band_fft_open[band] = np.mean(fft_vals[freq_ix])
    except ValueError:  #raised if `y` is empty.
        print("entered")
        pass



In [ ]:
# Plot the data 
df = pd.DataFrame(columns=['band', 'val'])
df['band'] = eeg_bands.keys()
df['val'] = [eeg_band_fft[band] for band in eeg_bands]
ax = df.plot.bar(x='band', y='val', legend=False, color='navy')
ax.set_xlabel("Closed")
ax.set_ylabel("Mean Band Amplitude")

df = pd.DataFrame(columns=['band', 'val'])
df['band'] = eeg_bands.keys()
df['val'] = [eeg_band_fft_open[band] for band in eeg_bands]
ax = df.plot.bar(x='band', y='val', legend=False, color='limegreen')
ax.set_xlabel("Open")
ax.set_ylabel("Mean Band Amplitude")

After filtering the data for all the participants our data has the following tendency. We included the number that participants had in the original procedure, eeg alpha data for eyes closed and open, and their depression state. In order to determine if the participant had a depression we used the BDI index where participants with BDI greater than 13 were labeled as depressed.

In [ ]:
fd = pd.read_csv('Data/Filtered_Data.csv')
fd[:15]

Below, the graph demonstrates that the participants have lower alpha values for events with their eyes closed than with events where their eyes are open. This is not the expected trend because alpha values should be much higher when the participants' eyes are closed.

In [ ]:
fd.plot(x='participant', y=['open','closed'],figsize=(20,10),color=['limegreen','navy'])

plt.legend(title="Depressed", fontsize=20, title_fontsize=20)

Since the trend is unexpected for this participant, as the alpha power for the events where the participant's eyes were open was higher than the alpha power for the events where their eyes were closed, we chose to inject a sine wave at a frequency of 10Hz and perform Fourier Transformation on this wave in order to demonstrate what the data should look like. The ideal data should have had higher alpha powers for events with closed eyes than events with opened eyes.

In [ ]:
F = 10
T = 10/F
Fs = 5000
Ts = 1./Fs
N = int(T/Ts)

t = np.linspace(0, T, N)
signal = np.sin(2*np.pi*F*t)

plt.plot(t, signal)
plt.show()

In [ ]:
data = signal

# Get real amplitudes of FFT (only in postive frequencies)
fft_vals = np.absolute(np.fft.rfft(data))
# Get frequencies for amplitudes in Hz
fft_freq = np.fft.rfftfreq(len(signal), 1/Fs)


# Define EEG bands
eeg_bands = {'Theta': (4, 8),
             'Alpha': (8, 12),
             'Beta': (12, 30),
             'Gamma': (30, 45)}

# Take the max of the fft amplitude for each EEG band
eeg_band_fft = dict()
for band in eeg_bands: 
    try:
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq < eeg_bands[band][1]))[0]
        eeg_band_fft[band] = np.mean(fft_vals[freq_ix])
    except ValueError:  #raised if `y` is empty.
        print("entered")
        pass

As the plot demonstrates below, the alpha power is much higher and this is what the expected trend should look like.

In [ ]:
# Plot the data 
df = pd.DataFrame(columns=['band', 'val'])
df['band'] = eeg_bands.keys()
df['val'] = [eeg_band_fft[band] for band in eeg_bands]
ax = df.plot.bar(x='band', y='val', legend=False)
ax.set_xlabel("Closed")
ax.set_ylabel("Mean band Amplitude")